In [1]:
import pandas as pd

In [2]:
BTC_data = pd.read_csv('Data/Altcoins/BTC.csv')
BTC_data.head()

,date,open,high,low,close,volume,% daily gain/loss,avg daily price,usd traded
0,2017-02-10,987.431,1010.68,920.00,987.100,64033.664983,-0.033521,965.34,6.181426e+07
1,2017-02-11,995.490,1028.28,964.66,992.310,27810.748059,-0.319441,996.47,2.771258e+07
2,2017-02-12,992.310,1022.89,983.50,1000.730,13861.128046,0.848525,1003.20,1.390548e+07
3,2017-02-13,1001.590,1021.50,958.88,1000.360,28432.244497,-0.122805,990.19,2.815332e+07
4,2017-02-14,1000.740,1033.17,974.51,992.907,38062.882105,-0.782721,1003.84,3.820904e+07
